In [ ]:
%pip install -r ../../requirements.txt

In [2]:
import os 
import sys
import json
from types import SimpleNamespace
import random
from glob import glob

import torch
import torchaudio
import torch.nn as nn
import numpy as np
import pandas as pd
import lightning as L
from lightning.pytorch.tuner import Tuner
from tqdm import tqdm
from timm import create_model, list_models
from torch.utils.data import DataLoader

sys.path.append('../../src')

from Raffaela_custom.data import AudioDataset, DataModule
from Raffaela_custom.trainer import TrainModule
from Raffaela_custom.net import SimpleCNN
from Raffaela_custom.utils import batch_to_device, get_min_max
from Raffaela_custom.eval import inference_k_random, error_analysis, inference_all

%load_ext autoreload
%autoreload 2

full_path = '../../'

In [5]:
cfg = SimpleNamespace()

cfg.data_path = f'{full_path}/data/production_data/6s_crop/'
cfg.n_classes = 66
cfg.pretrained = True
cfg.backbone = 'tf_efficientnetv2_s.in21k'
cfg.in_chans = 1
cfg.num_workers = 4
cfg.include_val = True

# Training Hyperparameters
cfg.n_epochs = 5
cfg.lr = 0.000251
cfg.weight_decay = 1e-4
cfg.label_smoothing = 0.1
cfg.batch_size = 32

# Mel Spectogram Hyperparameters
cfg.wav_crop_len = 5
cfg.sample_rate = 44100
cfg.n_mels = 128
cfg.n_fft = 2048
cfg.fmin = 300
cfg.fmax = cfg.sample_rate / 2 
cfg.window_size = cfg.n_fft
cfg.hop_length = int(cfg.n_fft / 2)
cfg.power = 2
cfg.top_db = 80.0

# Augmentation Parameters
cfg.impulse_prob = 0.2
cfg.noise_prob = 0.2
cfg.max_noise = 0.04
cfg.min_snr = 5
cfg.max_snr = 20
cfg.min, cfg.max = get_min_max(cfg, DataModule, SimpleCNN)
#cfg.min = -32.4444
#cfg.max = 54.2951

In [6]:
# Loss Function and class weights
class_weights = np.load(f'{full_path}/data/class_weights.npy')
loss_fn = nn.CrossEntropyLoss(weight=torch.from_numpy(class_weights).to('cuda'), 
                              label_smoothing=cfg.label_smoothing)

# Data Logic, Loading, Augmentation
dm = DataModule(cfg=cfg)

# Network
model = SimpleCNN(cfg)

# Training Logic
tmod = TrainModule(model, 
                   loss_fn=loss_fn, 
                   optimizer_name='Adam', 
                   optimizer_hparams={"lr": cfg.lr, "weight_decay": cfg.weight_decay},
                   cfg=cfg)

trainer = L.Trainer(
    max_epochs=20,
    accelerator="auto",
    devices="auto",
    enable_checkpointing=True,
    reload_dataloaders_every_n_epochs=False,
)

In [7]:
tuner = Tuner(trainer)
lr_finder = tuner.lr_find(model=tmod, datamodule=dm, min_lr=1e-7, max_lr=1e-2, num_training=100, attr_name="cfg")
fig = lr_finder.plot(suggest=True)
fig.show()

# Pick point based on plot, or get suggestion
new_lr = lr_finder.suggestion()

In [8]:
print(new_lr)